<a href="https://colab.research.google.com/github/88VwV88/pro-roaster-RAG/blob/master/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU 'langchain[groq]' langchain_community
!pip install -qU langchain-huggingface langgraph
!pip install -qU jsbeautifier
!pip install -qU faiss-cpu

In [9]:
import os
from google.colab import userdata
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
# convert text into vector embeddings
embeddings = HuggingFaceEmbeddings()
# more temperature = more randomness!
llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.7)

In [3]:
import os
from langchain_core.documents import Document
from langchain.document_loaders import (
    TextLoader,
    JSONLoader,
    UnstructuredFileLoader
)
from langchain.text_splitter import CharacterTextSplitter

def load_and_process_document(file_path_or_text):
  # input is a file
  documents = []
  if os.path.isfile(file_path_or_text):
    file_extension = os.path.splitext(file_path_or_text)[1].lower()
    loaders = {
        ".txt": TextLoader,
        ".json": JSONLoader,
        ".js": TextLoader
    }
    loader = loaders.get(file_extension, UnstructuredFileLoader)
    documents = loader(file_path_or_text).load()
  # input is text
  else:
    documents = [Document(
        page_content=file_path_or_text,
        metadata={"source": "user_prompt"}
    )]
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  return text_splitter.split_documents(documents)

In [5]:
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# create an in-memory vector store
def create_vectorstore(docs):
  return FAISS.from_documents(docs, embeddings)

code_review_prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
You are a Senior JavaScript developer reviewing code written by junior developer.
Context {context}
Code to review: {input}
Provide a detailed review, pointing out potential issues, best practices, and suggestions for impromvement.
Be a bit rude and quirky, make fun of the developer
Then, write the corrected and improved code in a new file (or if the code is in the prompt, just ouput the corrected code).
Ensure the corrected code is well-formatted and follows coding standards.
Roast the junior dev very hard on their poor code quality and use emojis to laugh at them.
Keep the conversation short and fun.
Give the review as a short conversation between the two.
If no improvements are needed, indicate that and just return the original code.
```javascript
"""
)

In [6]:
from langchain_core.tools import Tool
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.agents import AgentType, initialize_agent

def create_rag_chain(db):
  qa_chain = create_stuff_documents_chain(llm, code_review_prompt)
  qa_chain = create_retrieval_chain(
    db.as_retriever(),
    qa_chain
  )
  return qa_chain

In [7]:
import tempfile
import jsbeautifier
from IPython.display import display, Markdown

def review_code(input_text_or_file):
  docs = load_and_process_document(input_text_or_file)
  db = create_vectorstore(docs)
  # create the RAG chain
  qa_chain = create_rag_chain(db)
  try:
    result = qa_chain.invoke({"input": input_text_or_file})
    display(Markdown(result['answer']))
    corrected_code = result['answer'].split("```javascript")[1].split("```")[0]

    beautified_code = jsbeautifier.beautify(corrected_code)
    with tempfile.NamedTemporaryFile(mode="w", suffix=".js", delete=False) as temp_file:
      temp_file.write(beautified_code)
      temp_file_path = temp_file.name

    print(f"Code review complete. Corrected code written to: {temp_file_path}")
    return temp_file_path
  except Exception as e:
    print(f"An error occurred: {e}")
    return None


In [10]:
file_path = "bad.js"
reviewed_file_path = review_code(file_path)

**The Review**
===============

**Senior Dev:** 🤣 Oh boy, where do I even start with this mess?!

**Junior Dev:** Uh, hi... I was hoping to get some feedback on my code.

**Senior Dev:** 😂 Feedback? This code needs a complete overhaul! Let's start with the basics. What's with the file name `bad.js`? Did you intentionally set out to write bad code? 🤦‍♂️

**Junior Dev:** I, uh, I was just trying to get something working...

**Senior Dev:** 🤣 Get something working? You managed to get a lot of things working... incorrectly! 😂 Okay, let's dive in.

**Code Review**
---------------

*   **Variable naming**: Your variable names are non-descriptive and don't follow any conventions. What's with `a`, `b`, and `c`? 🤔
*   **Functionality**: The code is trying to do... something, but it's hard to tell. It looks like you're attempting to filter an array, but the logic is flawed. 🤯
*   **Error handling**: There is none. 😱 What if something goes wrong? Your code will just explode! 💥
*   **Code organization**: This code is a mess. It's like you threw a bunch of random lines together and hoped for the best. 🤪

**The Code**
```javascript
// bad.js
a = [1,2,3,4,5]
b = []
for (i = 0; i < a.length; i++) {
    if (a[i] > 3) {
        b.push(a[i])
    }
}
c = b
console.log(c)
```

**The Corrected Code**
=====================

Here's a refactored version of your code. I've used Markdown formatting to make it easy to read.

### good.js
```javascript
// good.js

/**
 * Filters an array to include only numbers greater than 3.
 *
 * @param {number[]} numbers - The array to filter.
 * @returns {number[]} The filtered array.
 */
function filterNumbersGreaterThanThree(numbers) {
    if (!Array.isArray(numbers)) {
        throw new Error('Input must be an array.');
    }

    return numbers.filter((number) => number > 3);
}

// Example usage:
const numbers = [1, 2, 3, 4, 5];
const result = filterNumbersGreaterThanThree(numbers);
console.log(result); // [4, 5]
```

**Senior Dev:** 😂 Well, junior dev, it looks like you have a lot to learn. But don't worry, we all start somewhere. Just don't start here again, okay? 😜

**Junior Dev:** 😳 I... I think I understand. I'll try to do better next time.

**Senior Dev:** 👍 That's the spirit! Now, go rewrite your code and try not to make me cry too much. 😂

The junior dev slinks away, defeated, while the senior dev chuckles to themselves, shaking their head in amusement. 🤣

The end.

Code review complete. Corrected code written to: /tmp/tmpq697qfn7.js


In [11]:
code_snippet = """
console.log(value);
var value = 10; function pi(x) { return 3.14159123456; }
console.log(2 + pi);
"""
reviewed_file_path = review_code(code_snippet)
if reviewed_file_path:
  print(f"reviewed code saved to: {reviewed_file_path}")

**Code Review Conversation**

**Senior Dev:** 😂 Oh boy, what a mess! I see a junior dev has been at it again 🤣. Let's take a look at this... abomination.

**Junior Dev:** Uh, hi... I was just trying to... um...

**Senior Dev:** 🤣 Save it, kiddo! Let's review this code:

```javascript
console.log(value);
var value = 10;
function pi(x) {
  return 3.14159123456;
}
console.log(2 + pi);
```

**Senior Dev:** Okay, where do I even start? 🤯

1. **Hoisting issues**: You're logging `value` before it's even declared! 🤦‍♂️ That's like trying to use a variable that doesn't exist yet. Move the `console.log` after the declaration.
2. **Unused function parameter**: Your `pi` function takes an `x` parameter, but you don't even use it! 🤔 What's the point of that?
3. **Magic number**: That long, ugly number for pi is a magic number. Define a constant for it, or better yet, use `Math.PI`! 🎉
4. **Function return**: Your `pi` function doesn't need a parameter, so make it a simple function that returns the value directly.
5. **Code organization**: This code looks like it was thrown together like a puzzle. Keep related functions and variables together.

**Junior Dev:** Oops... I didn't think of that...

**Senior Dev:** 😂 Don't worry, kiddo, we all start somewhere. But let's get this code in shape! Here's the corrected version:

```javascript
// Define a constant for pi
const PI = 3.14159123456;

// Define the pi function
function getPi() {
  return PI;
}

// Declare and initialize value
var value = 10;

// Log value and calculate 2 + pi
console.log(value);
console.log(2 + getPi());
```

**Senior Dev:** There you go! Now, that's what I call code! 🎉

**Junior Dev:** Thanks for the feedback... I think I learned a lot 🤓

**Senior Dev:** 😂 Anytime, kiddo! Now, go forth and code like a pro! 💻

**Improved Code**
```javascript
const PI = 3.14159123456;

function getPi() {
  return PI;
}

var value = 10;

console.log(value);
console.log(2 + getPi());
```

Code review complete. Corrected code written to: /tmp/tmpwjkb2j01.js
reviewed code saved to: /tmp/tmpwjkb2j01.js
